In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp '/content/drive/MyDrive/Colab Notebooks/Hackathon_LV2_ONLINE#2/Medical AI  OCR Hackathon/Liver Lesion/liver-ultrasound-detection.zip' '/content/liver-ultrasound-detection.zip'

In [ ]:
!cp '/content/drive/MyDrive/Colab Notebooks/Hackathon_LV2_ONLINE#2/Medical AI  OCR Hackathon/Liver Lesion/mapping_withPATH.csv' '/content/mapping_withPATH.csv'

In [ ]:
!unzip /content/liver-ultrasound-detection.zip && rm -rf /content/liver-ultrasound-detection.zip

# Ultrasound test

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def display_image(image, title, cmap='gray'):
    """Displays an image using matplotlib."""
    plt.imshow(image, cmap=cmap)
    plt.title(title)
    plt.axis('off')
    plt.show()

# Load the image
image_path = '/content/test/test/images/10001.jpg' # ultrasound
image = cv2.imread(image_path)
display_image(image, 'image')

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
display_image(gray, 'gray')


# Apply binary threshold
ret, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY )# for ultrasound

# Create a kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Apply opening operation to remove small noise
opened_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

# Find all connected components
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(opened_mask, connectivity=8)

# Find the largest connected component excluding the background
largest_component = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
largest_mask = np.zeros_like(opened_mask)
largest_mask[labels == largest_component] = 255

# Fill holes using flood fill
im_floodfill = largest_mask.copy()
h, w = im_floodfill.shape[:2]
mask_floodfill = np.zeros((h+2, w+2), np.uint8)
cv2.floodFill(im_floodfill, mask_floodfill, (0,0), 255)
filled_mask = cv2.bitwise_or(largest_mask, cv2.bitwise_not(im_floodfill))

# Apply the mask to the original image
masked_image = cv2.bitwise_and(image, image, mask=filled_mask)
display_image(masked_image, 'Masked Image', cmap=None)

-----------

# Generated mobile-like test

In [ ]:
# Load the image
image_path = '/content/102890_fake.png' #generated mobile-like
image = cv2.imread(image_path)
display_image(image, 'image')

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
display_image(gray, 'gray')

# For generated images
adapthresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                          cv2.THRESH_BINARY, 199, 2)
display_image(adapthresh, 'adaptive thresh')

ret, mask = cv2.threshold(adapthresh, 0, 255, cv2.THRESH_BINARY )# for generated mobile-like

# Create a kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Apply opening operation to remove small noise
opened_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

# Find all connected components
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(opened_mask, connectivity=8)

# Find the largest connected component excluding the background
largest_component = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
largest_mask = np.zeros_like(opened_mask)
largest_mask[labels == largest_component] = 255

# Fill holes using flood fill
im_floodfill = largest_mask.copy()
h, w = im_floodfill.shape[:2]
mask_floodfill = np.zeros((h+2, w+2), np.uint8)
cv2.floodFill(im_floodfill, mask_floodfill, (0,0), 255)
filled_mask = cv2.bitwise_or(largest_mask, cv2.bitwise_not(im_floodfill))

# Smooth the left edge
contours, _ = cv2.findContours(filled_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for contour in contours:
    if cv2.contourArea(contour) > 500:  # Only consider significant contours
        epsilon = 0.01 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        hull = cv2.convexHull(approx)
        cv2.drawContours(filled_mask, [hull], 0, (255), thickness=cv2.FILLED)


# Apply the mask to the original image
masked_image = cv2.bitwise_and(image, image, mask=filled_mask)
display_image(masked_image, 'Masked Image', cmap=None)

-----------

# Mobile test

In [ ]:
# img path
# image_path = '/content/test/test/images/10001.jpg' # ultrasound
image_path = '/content/102890_fake.png' #generated mobile-like
# image_path = '/content/1060.jpg' # mobile image

image = cv2.imread(image_path)
display_image(image, 'Original Image', cmap=None)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# display_image(gray, 'Gray Image')

# For generated images
ret,th1 = cv2.threshold(gray, np.mean(image),255 ,cv2.THRESH_BINARY)
# ret,th1 = cv2.threshold(gray, 90,255 ,cv2.THRESH_BINARY)
display_image(th1, 'glob thresh')

# Fill holes using flood fill
im_floodfill = th1.copy()
h, w = im_floodfill.shape[:2]
mask_floodfill = np.zeros((h+2, w+2), np.uint8)
cv2.floodFill(im_floodfill, mask_floodfill, (0,0), 255)
filled_mask = cv2.bitwise_or(th1, cv2.bitwise_not(im_floodfill))

# Smooth the left edge
contours, _ = cv2.findContours(filled_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for contour in contours:
    if cv2.contourArea(contour) > 500:  # Only consider significant contours
        epsilon = 0.01 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        hull = cv2.convexHull(approx)
        cv2.drawContours(filled_mask, [hull], 0, (255), thickness=cv2.FILLED)

flipped_mask = cv2.flip(filled_mask, 1)
display_image(flipped_mask, 'flipped_mask')

sum_mask = filled_mask + flipped_mask
display_image(sum_mask, 'sum_mask')


# Fill holes using flood fill
im_floodfill = sum_mask.copy()
h, w = im_floodfill.shape[:2]
mask_floodfill = np.zeros((h+2, w+2), np.uint8)
cv2.floodFill(im_floodfill, mask_floodfill, (0,0), 255)
filled_mask = cv2.bitwise_or(sum_mask, cv2.bitwise_not(im_floodfill))


# Optionally apply opening to remove small noise
cleaned_mask = cv2.morphologyEx(filled_mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
display_image(cleaned_mask, 'cleaned_mask')


# Find contours
contours, _ = cv2.findContours(cleaned_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create an empty mask
hull_mask = np.zeros_like(gray, dtype=np.uint8)

# Draw the convex hull of the largest contour on the mask
if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    hull = cv2.convexHull(largest_contour)
    cv2.drawContours(hull_mask, [hull], -1, 255, thickness=cv2.FILLED)

display_image(hull_mask, 'Convex Hull Mask')


# # Apply the mask to the original image
masked_image = cv2.bitwise_and(image, image, mask=hull_mask)
display_image(masked_image, 'Masked Image', cmap=None)

------
# Prepare function

In [ ]:
def extract_ultrasound(image, imtype='mobile'):
  if imtype == 'mobile':
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    min_thresh = np.mean(image) + 0.3*np.std(image)
    max_thresh = 255
    print(min_thresh, max_thresh)
    ret,th1 = cv2.threshold(gray , min_thresh,  max_thresh, cv2.THRESH_BINARY)
    im_floodfill = th1.copy()
    h, w = im_floodfill.shape[:2]
    mask_floodfill = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(im_floodfill, mask_floodfill, (0,0), 255)
    filled_mask = cv2.bitwise_or(th1, cv2.bitwise_not(im_floodfill))

    # Smooth the left edge
    contours, _ = cv2.findContours(filled_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Only consider significant contours
            epsilon = 0.01 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            hull = cv2.convexHull(approx)
            cv2.drawContours(filled_mask, [hull], 0, (255), thickness=cv2.FILLED)

    flipped_mask = cv2.flip(filled_mask, 1)
    sum_mask = filled_mask + flipped_mask
    im_floodfill = sum_mask.copy()
    h, w = im_floodfill.shape[:2]
    mask_floodfill = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(im_floodfill, mask_floodfill, (0,0), 255)
    filled_mask = cv2.bitwise_or(sum_mask, cv2.bitwise_not(im_floodfill))
    cleaned_mask = cv2.morphologyEx(filled_mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
    contours, _ = cv2.findContours(cleaned_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    hull_mask = np.zeros_like(gray, dtype=np.uint8)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        hull = cv2.convexHull(largest_contour)
        cv2.drawContours(hull_mask, [hull], -1, 255, thickness=cv2.FILLED)
    masked_image = cv2.bitwise_and(image, image, mask=hull_mask)

  elif imtype == 'machine':
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY )# for ultrasound
    kernel = np.ones((5, 5), np.uint8)
    opened_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(opened_mask, connectivity=8)
    largest_component = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
    largest_mask = np.zeros_like(opened_mask)
    largest_mask[labels == largest_component] = 255
    im_floodfill = largest_mask.copy()
    h, w = im_floodfill.shape[:2]
    mask_floodfill = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(im_floodfill, mask_floodfill, (0,0), 255)
    filled_mask = cv2.bitwise_or(largest_mask, cv2.bitwise_not(im_floodfill))
    masked_image = cv2.bitwise_and(image, image, mask=filled_mask)

  else: #Gital's generated US

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    adapthresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                              cv2.THRESH_BINARY, 199, 2)
    ret, mask = cv2.threshold(adapthresh, 0, 255, cv2.THRESH_BINARY )# for generated mobile-like
    kernel = np.ones((5, 5), np.uint8)
    opened_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(opened_mask, connectivity=8)
    largest_component = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
    largest_mask = np.zeros_like(opened_mask)
    largest_mask[labels == largest_component] = 255
    im_floodfill = largest_mask.copy()
    h, w = im_floodfill.shape[:2]
    mask_floodfill = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(im_floodfill, mask_floodfill, (0,0), 255)
    filled_mask = cv2.bitwise_or(largest_mask, cv2.bitwise_not(im_floodfill))
    contours, _ = cv2.findContours(filled_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Only consider significant contours
            epsilon = 0.01 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            hull = cv2.convexHull(approx)
            cv2.drawContours(filled_mask, [hull], 0, (255), thickness=cv2.FILLED)
    masked_image = cv2.bitwise_and(image, image, mask=filled_mask)

  return masked_image

In [ ]:
image_path = '/content/val/val/images/1001.jpg'
# image_path = '/content/1060.jpg' # mobile image

image = cv2.imread(image_path)
display_image(image, 'Original Image', cmap=None)

test = extract_ultrasound(image, imtype='mobile')
display_image(test, 'test', cmap=None)

In [ ]:
image_path = '/content/test/test/images/10001.jpg' # ultrasound
# image_path = '/content/1060.jpg' # mobile image

image = cv2.imread(image_path)
display_image(image, 'Original Image', cmap=None)

test = extract_ultrasound(image, imtype='machine')
display_image(test, 'test', cmap=None)

In [ ]:
image_path = '/content/102890_fake.png' #Gital's generated mobile-like


image = cv2.imread(image_path)
display_image(image, 'Original Image', cmap=None)

test = extract_ultrasound(image, imtype='a')
display_image(test, 'test')